In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

examples = [
    {"input": "탑건", "output": "🛩️👨‍✈️🔥"},
    {"input": "대부", "output": "👨‍👨‍👦🔫🍝"},
]
example_template = """
영화 제목: {input}
이모티콘: {output}
"""
few_shot_prompt = FewShotPromptTemplate.from_examples(
    examples=examples,
    example_prompt=PromptTemplate.from_template(example_template),
    prefix="다음은 영화 제목을 세 개의 이모티콘으로 표현한 예시입니다.",
    suffix="영화 제목: {question}\n이모티콘:",
    input_variables=["question"],
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=few_shot_prompt,
    verbose=True,
)

response_1 = chain.predict(question="탑건")
print("탑건:", response_1)

response_2 = chain.predict(question="대부")
print("대부:", response_2)

response_3 = chain.predict(question="내가 처음에 물어본 영화가 뭐였지?")
print("처음 질문한 영화:", response_3)


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = TextLoader("./document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings,cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

memory = ConversationSummaryBufferMemory(memory_key="chat_history")

retriever = vectorstore.as_retriever()


final_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """Given the following extracted parts of a long document and a question, create a final answer.
        If you don't know the answer, just say that you don't knoe. Don't try to make up an answer.
        -----
        {context}
        """,
    ),
    ("human","{question}")
])

map_chain = { "documents": retriever, "question": RunnablePassthrough() } | final_prompt | llm

chain = (
    {
        "context": map_chain, 
        "question": RunnablePassthrough(), 
    } 
    | final_prompt 
    | llm
)


def ask_question(question):
    context = retriever.get_relevant_documents(question)
    context_text = "\n".join([doc.page_content for doc in context])
    
    # 대화 기록에 문맥과 질문 추가
    memory.buffer.append(f"Context: {context_text}")
    memory.buffer.append(f"Question: {question}")
    
    # 체인 실행
    result = chain.invoke({"context": context_text, "question": question})
    
    # 대화 기록에 결과 추가
    memory.buffer.append(f"Answer: {result}")
    
    return result

# 13. 예시 질문들
questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]

for question in questions:
    print(f"Q: {question}")
    answer = ask_question(question)
    print(f"A: {answer}")



SyntaxError: invalid syntax (2595496057.py, line 30)